In [ ]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import xarray as xr
import pandas as pd
import scipy.io as sio
import sys
import os
import glob
import copy
import pathlib as pth
import importlib
import math
sys.path.append('/home/jleland/Coding/Projects/flopter')
import flopter.spice.splopter as spl
import flopter.spice.tdata as td
import flopter.core.ivdata as iv
import flopter.core.fitters as fts
import flopter.core.fitdata as fd
import flopter.core.lputils as lpu
import flopter.core.constants as c
import flopter.spice.inputparser as inp
import flopter.spice.normalise as nrm
import flopter.spice.xrfuncs as xrf

In [2]:
v_dum = np.linspace(-30,0,500)

In [3]:
fig, ax = plt.subplots(2)

e_current = - np.exp(v_dum+2.4398)
ax[0].plot(v_dum, e_current)

exp_fitter = fts.ExperimentalIVFitter()
ax[1].plot(v_dum, exp_fitter.fit_function(v_dum, *[-1,0.02,1,-2.5]))

In [4]:
I_0  = -75.0
a = 0.02
T_e = 1
v_f = -2.5

In [5]:
v = v_dum
mod_v_fl = a * np.float_power(np.absolute(v_f / T_e), [0.75])

# fig, ax = plt.subplots()
# ax.plot(v_dum, mod_v_fl)
print(mod_v_fl)

[0.03976354]


In [6]:
f = (1 + mod_v_fl) / (np.exp(v_f/T_e) + mod_v_fl)

# fig, ax = plt.subplots()
# ax.plot(v_dum, f)
print(f)

[8.5332461]


In [7]:
fig, ax = plt.subplots(2)
# np.where(v <= v_f, (     , 0)))
I_i = I_0 * (1 + (a * np.where(v <= 0, (np.float_power(np.absolute(v / T_e), [0.75])), 0)))
I_e = I_0 * f * np.exp(v / T_e) * (1 + (a * np.where(v <= 0, (np.float_power(np.absolute(v / T_e), [0.75])), 0)))
# ax[0].plot(v, I_i)
ax[0].plot(v, I_e)

ax[1].plot(v, I_i - I_e)

for axis in ax:
    axis.axvline(x=v_f, **c.AX_LINE_DEFAULTS)

In [8]:
fig, ax = plt.subplots()
ax.axvline(x=v_f, **c.AX_LINE_DEFAULTS)

I_e1 = I_0 * f * np.exp(v / T_e)
I_e2 = I_0 * f * np.exp(v / T_e) * a * (np.where(v <= 0, (np.float_power(np.absolute(v / T_e), [0.75])), 0))
ax.plot(v, I_e1)
ax.plot(v, I_e2)

## Alternate, multiplicative form

In [9]:
I_0  = -21.452
a = 0.009
T_e = 1.052
v_f = -2.345

In [10]:
V = (v - v_f) / T_e

# I_i = I_0 * (1 + (a * np.where(v <= v_f, (np.float_power(np.absolute(V), [0.75])), 0)))
# I_e = I_0 * np.exp(V) * (1 + np.where(v <= v_f, (np.float_power(np.absolute(V), [0.75])), 0))

I_i = I_0 * (1 + (a * np.float_power(np.absolute(V), [0.75])))
I_e = I_0 * np.exp(V) * (1 + (a * np.float_power(np.absolute(V), [0.75])))


fig, ax = plt.subplots(2, sharey=True)
for axis in ax:
    axis.axvline(x=v_f, **c.AX_LINE_DEFAULTS)
    axis.set_xlim(-30, -2.0)
    axis.set_ylim(-100, 100.0)

ax[0].plot(v, I_i)
ax[0].plot(v, I_e)
ax[1].plot(v, I_i - I_e)



## Try it on an IV curve

In [11]:
spl_path = pth.Path('/home/jleland/data/external_big/spice/marconi/spice2/sheath_exp_hg/flat_flush_as/alpha_yz_-10.0')
print(spl_path)
spl_backup = list(spl_path.glob('backup*'))[-1]

/home/jleland/data/external_big/spice/marconi/spice2/sheath_exp_hg/flat_flush_as/alpha_yz_-10.0


In [12]:
non_standard_variables = {'t', 'ProbePot', 'npartproc', 'Nz', 'Nzmax', 'Ny', 'count', 'Npc', 'snumber', 'nproc'}
desired_variables = (td.DEFAULT_REDUCED_DATASET | non_standard_variables) - {td.OBJECTSCURRENTFLUXE, td.OBJECTSCURRENTFLUXI}

In [13]:
splopter =  spl.Splopter(spl_backup, reduce=desired_variables, ignore_tzero_fl=True, version=2.14,
                         store_dataframe_fl=True)
print(splopter.tdata.dt)

parser = inp.InputParser(input_filename=spl_backup / 'input.inp')
denormaliser = nrm.Denormaliser(dt=splopter.tdata.dt, input_parser=parser) 

Spice data directory is not valid, attempting to auto-fix.
Passed Spice directory (/home/jleland/data/external_big/spice/marconi/spice2/sheath_exp_hg/flat_flush_as/alpha_yz_-10.0/backup_20201222-0127) doesn't seem to be valid.
Continuing anyway.


/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/scipy/io/matlab/mio.py:218: MatReadWarning: Duplicate variable name "npartproc" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)


[[0.00016981]]
dx = 1.0, dy = 0.5, dz = 0.5


In [14]:
# Test
iv_df, iv_data = xrf.homogenise_run(spl_backup)


Spice data directory is not valid, attempting to auto-fix.
Passed Spice directory (/home/jleland/data/external_big/spice/marconi/spice2/sheath_exp_hg/flat_flush_as/alpha_yz_-10.0/backup_20201222-0127) doesn't seem to be valid.
Continuing anyway.


In [15]:
importlib.reload(fts)

<module 'flopter.core.fitters' from '/home/jleland/coding/projects/flopter/flopter/core/fitters.py'>

In [16]:
calced_a = lpu.calc_new_sheath_expansion_param(T_e, 1e18, 5e-3, 1e-3, np.radians(10), 0.0, 0.0, 
                                               c_1=1.4, c_2=0.39)
print(calced_a)

0.011014489099584407


In [17]:
fig, ax = plt.subplots()

iv_region = np.where((iv_data['V'] <= -2.3) & (iv_data['V'] >= -30))
fit_iv_data = iv.IVData.non_contiguous_trim(iv_data, iv_region)



exp_fitter = fts.ExperimentalIVFitter()
iv_fitter = fts.FullIVFitter()
# exp_fitter.set_fixed_values({'a':calced_a})
# iv_fitter.set_fixed_values({'a':calced_a})

iv_fitter.fit(fit_iv_data['V'], fit_iv_data['I']).plot(ax=ax)
exp_fitter.fit(fit_iv_data['V'], fit_iv_data['I']).plot(ax=ax)

# ax.plot(v_dum, exp_fitter.fit_function(v_dum, *[-1,0.02,1,2.5]))

In [18]:
fit_iv_data['sigma']

array([0.89142317, 0.94052106, 1.06418549, 1.10365798, 0.92085086,
       0.9020483 , 0.98397126, 0.94013196, 0.93190598, 0.98318621,
       1.06327079, 1.05895905, 1.01437953, 1.15147743, 0.96436029,
       0.88479904, 1.00543307, 0.91772708, 0.98257675, 0.98852605,
       0.93527367, 0.98186699, 0.8638583 , 1.07257747, 0.9263594 ,
       1.06895566, 0.93981393, 0.9397599 , 0.95225349, 1.03931378,
       0.9656078 , 0.87769503, 0.96850686, 0.91315916, 0.950895  ,
       0.8729864 , 0.92858946, 0.98690877, 1.06909386, 0.86417756,
       1.01932391, 1.105567  , 0.95410253, 0.9015325 , 1.03721607,
       1.03293413, 0.82913253, 0.98153309, 1.05024259, 0.9911846 ,
       0.97944125, 0.92234496, 1.08180089, 0.98378356, 0.98759036,
       1.01000878, 0.87673848, 0.98263783, 0.97503818, 0.7974379 ,
       0.96008401, 0.98021047, 0.92252389, 0.92187432, 0.89794446,
       0.98033645, 0.98096849, 1.00792795, 1.03985669, 1.10565595,
       0.87430225, 1.05403415, 0.9557717 , 1.05722135, 0.87086

In [19]:
fig, ax = plt.subplots()
# ax.plot(fit_iv_data['V'], fit_iv_data['I_e'])

initial_vals = [T_e, v_f, a]

ese_fitter = fts.ElectronCurrentSEFitter()
ese_fitter.set_fixed_values({'T_e':1.15})
e_fitter = fts.ElectronCurrentFitter()

e_fitter.fit(iv_data['V'], iv_data['I_e']).plot(ax=ax)

ese_fitter.fit(iv_data['V'], iv_data['I_e']).plot(ax=ax)


In [20]:
fig, ax = plt.subplots()

params = [-20.0, 0.02, 10.0, -2.5]
print(params)

dummy_v = np.linspace(-200,-2.5,1000)
iv_I = iv_fitter.fit_function(dummy_v, *params)
exp_I = exp_fitter.fit_function(dummy_v, *params)

ax.plot(dummy_v, iv_I)
ax.plot(dummy_v, exp_I)

[-20.0, 0.02, 10.0, -2.5]


## Comparison of SE Models

In [21]:
cb_palette = ['#4477AA', '#66CCEE', '#228833', '#CCBB44', 
              '#EE6677', '#AA3377', '#BBBBBB']
contrast_palette = ['#FFFFFF', '#DDAA33', '#BB5566', '#004488', '#000000']

In [207]:
importlib.reload(lpu)

<module 'flopter.core.lputils' from '/home/jleland/coding/projects/flopter/flopter/core/lputils.py'>

In [227]:
def plot_se_comparison(ax=None):
    if ax is None:
        fig, ax = plt.subplots()


    dummy_theta = np.linspace(0.0001, 45.01, 5000)
    T_e_real = 5
    n_0 = 1e18
    L = 300
    g = 60
    d_perp = 3e-4
    lambda_D = lpu.debye_length(T_e_real, n_0)
    print(lambda_D)

    MS_a = lpu.calc_sheath_expansion_param(
        T_e_real, n_0, L * lambda_D, 0 , np.radians(dummy_theta), 
        c_1=0.5, c_2=0.6,
    )
    ax.errorbar(dummy_theta, MS_a, label=r'Bergmann', fmt='-', 
                color=contrast_palette[4], linewidth=1.0, alpha=0.6)


    B_a = lpu.calc_sheath_expansion_param(
        T_e_real, n_0, L * lambda_D, g * lambda_D, np.radians(dummy_theta), 
        c_1=0.5, c_2=0.6,
    )
    ax.errorbar(dummy_theta, B_a, label=r'Murphy-Sugrue', fmt='-', 
                color=contrast_palette[1], linewidth=1.0)


    L_a = lpu.calc_2d_box_sheath_expansion_param(
        T_e_real, n_0, L * lambda_D, g * lambda_D, np.radians(dummy_theta), 
        d_perp, np.radians(10.0),
    #     0, 0,
        c_1=0.5, c_2=0.6,
        delta_0=30 * lambda_D,
    )
    ax.errorbar(dummy_theta, L_a, label=r'Rotated Rectangle', fmt='-', 
                color=contrast_palette[2], linewidth=1.0)



    calced_a = lpu.calc_new_sheath_expansion_param(
        T_e_real, n_0, L * lambda_D, g * lambda_D, np.radians(dummy_theta), 
        d_perp, np.radians(10.0),
    #     0, 0,
    #     c_1=0.5, c_2=0.6,
    #     c_1=1.4, c_2=0.39, # from hg-theta=15-30
        c_1=0.5, c_2=0.6, # from hg-theta=11-30
    )
    ax.errorbar(dummy_theta, calced_a, label=r'Parallelogram', fmt='-', 
                color=contrast_palette[3], linewidth=1.0)


    ax.set_ylim(4e-3, 1e1)
    ax.set_xlim(0, 30.5)
    ax.set_yscale('log')
    ax.set_ylabel(r'$a$ [unitless]')
    ax.set_xlabel(r'$\theta$ [$^{\circ}$]')

    ax.legend()
#     fig.tight_layout()
    
plot_se_comparison()

1.6622799720325184e-05


In [233]:
def plot_rr_delta_0(ax=None, theta_p=10.0):
    if ax is None:
        fig, ax = plt.subplots()

    calced_a = lpu.calc_new_sheath_expansion_param(
        T_e_real, n_0, L * lambda_D, g * lambda_D, np.radians(dummy_theta), 
        d_perp, np.radians(theta_p),
        c_1=0.5, c_2=0.6,
    )
    ax.errorbar(dummy_theta, calced_a, label=r'Parallelogram', fmt='-', 
                color='k', linewidth=1.0)

    for i, del_0 in enumerate(np.linspace(0,125,6)):
        calced_a = lpu.calc_2d_box_sheath_expansion_param(
            T_e_real, n_0, L * lambda_D, g * lambda_D, np.radians(dummy_theta), 
            d_perp, np.radians(theta_p),
        #     0, 0,
        #     c_1=0.5, c_2=0.6,
        #     c_1=1.4, c_2=0.39, # from hg-theta=15-30
            c_1=0.5, c_2=0.6, # from hg-theta=11-30
            delta_0=(del_0*lambda_D)
        )
        ax.errorbar(dummy_theta, calced_a, label=f'{del_0}', fmt='-', 
                    color=cb_palette[i], linewidth=1.0)

    ax.set_ylim(5e-3, 1e-1)
    ax.set_xlim(0, 30.5)
    ax.set_yscale('log')
    ax.set_ylabel(r'$a$ [unitless]')
    ax.set_xlabel(r'$\theta$ [$^{\circ}$]')

    ax.legend(title=r'$\Delta_0 [\lambda_D]$')
#     fig.tight_layout()
    
plot_rr_delta_0(theta_p=10)

In [234]:
fig, ax = plt.subplots(1, 2, figsize=[10,5])

plot_se_comparison(ax[0])
plot_rr_delta_0(ax[1])
fig.tight_layout()

1.6622799720325184e-05


In [141]:
print(L * lambda_D * 1000) # L in mm
print(g * lambda_D * 1000) # g in mm

4.986839916097555
0.9973679832195111


In [30]:
theta = np.radians(10.0)
theta_p = np.radians(45.0)

def L_0_p(L, g, theta, theta_p, d_perp):
    return (L+g) * np.sin(theta) + (L*np.tan(theta_p) - d_perp) * np.cos(theta)

def L_0_rr(L, g, theta, theta_p, d_perp, delta_0=0.0):
    return ((L+g) * np.sin(theta) 
            + (L*np.tan(theta_p) - d_perp) * np.cos(theta)
            + (delta_0 * np.cos(theta) * (np.cos(theta_p)*(1 - np.tan(theta)*np.tan(theta_p)) - 1))
           )

print(f'Parallelogram: {L_0_p(L, g, theta, theta_p, d_perp)}')
print(f'Rotated Rectangle: {L_0_rr(L, g, theta, theta_p, d_perp, 10.0)}')


Parallelogram: 357.9553744214314
Rotated Rectangle: 353.84306125481976


In [31]:
delta_0 = 30.0

dummy_theta
dummy_theta_p = np.linspace(0, 20, 5)


fig, ax = plt.subplots()
for i, theta_probe in enumerate(dummy_theta_p):
    theta_p_rads = np.radians(theta_probe)
    ax.plot(dummy_theta, L_0_p(L, g, np.radians(dummy_theta), theta_p_rads, d_perp), 
            label=r'${}\degree$'.format(theta_probe), ls='dotted', color=cb_palette[i])
    ax.plot(dummy_theta, L_0_rr(L, g, np.radians(dummy_theta), theta_p_rads, d_perp, delta_0=delta_0),
            ls='-', color=cb_palette[i])
ax.legend(title=r'$\theta_p$')
ax.set_ylim(0, None)
ax.set_xlim(0, 45)
ax.set_ylabel(r'$L_0$')
ax.set_xlabel(r'$\theta$')

Text(0.5, 0, '$\\theta$')

In [167]:
v_f

-2.345

In [171]:
v_fl_est = 0.5*np.log(4*np.pi/1836)

In [179]:
def get_sheath_thickness(V, theta, lambda_D=1.0, T_e=1.0):
    return lambda_D * np.float_power(np.abs(V/T_e), 0.75) / np.sqrt(np.sin(theta))

In [244]:
dummy_phi = np.linspace(-5.5,-0.01,101)

fig, ax = plt.subplots()
for i, theta_probe in enumerate(dummy_theta_p):
    theta_p_rads = np.radians(theta_probe)
    delta_1 = get_sheath_thickness(dummy_phi, theta_p_rads)
    delta_0 = get_sheath_thickness(v_fl_est, theta_p_rads)
    ax.plot(dummy_phi, delta_0 + delta_1, 
            label=r'${}\degree$'.format(theta_probe), color=cb_palette[i])
ax.legend(title=r'$\theta$')

ax.set_ylim(0, None)
ax.set_xlim(-5.5, 0)
ax.set_ylabel(r'$\Delta$')
ax.set_xlabel(r'$V_w$')

/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in true_divide
  
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in double_scalars
  


Text(0.5, 0, '$V_w$')

## Simulation Shadow Calculations

In [98]:
def get_L_sh(voltage, theta, rho_L, c_1=2.0, c_2=0.97, g=60, h=0):
    theta_rads = np.radians(theta)
    delta_1 = get_Delta_1(voltage, theta, c_2=c_2)
    delta = get_delta(voltage, theta, c_1=c_1)
    return np.ceil((delta_1 + h + (rho_L * np.cos(theta_rads))) / np.tan(theta_rads) - (g - delta))

def get_Delta_1(voltage, theta, c_2=0.97):
    theta_rads = np.radians(theta)
    return (c_2 * np.float_power(np.abs(voltage), 0.75) / np.sqrt(np.sin(theta_rads)))

def get_delta(voltage, theta, c_1=2.0):
    theta_rads = np.radians(theta)
    return (c_1 * np.float_power(np.abs(voltage), 0.75) / np.sqrt(np.sin(theta_rads)))

In [41]:
dummy_theta_coarse = np.linspace(1.5,15,100)

fig, ax = plt.subplots()

for voltage in np.linspace(20, 10, 5):
    
    ax.plot(dummy_theta_coarse, get_sheath_height(voltage, dummy_theta_coarse), label=f'{voltage}')
ax.axhline(y=136, **c.AX_LINE_DEFAULTS)
ax.set_ylabel('Required window width')
ax.legend()

In [112]:
T_e = 5
n_0 = 1e18
B = 0.8


rho_l = lpu.ion_larmor_radius(T_e, 0.8)
c_s = lpu.sound_speed(T_e, gamma_i=(3/5))
lambda_D = lpu.debye_length(T_e, n_0)
xi = rho_l / lambda_D

g = int(1e-3 / lambda_D)
print(xi, g)

17.180450093286176 60


In [113]:
get_L_sh(15, 10.0, xi, g=g)

174.0

In [151]:
thetas = np.concatenate([np.arange(2, 13), np.arange(14, 21, 2), np.arange(25, 31, 5)])
-thetas

array([ -2,  -3,  -4,  -5,  -6,  -7,  -8,  -9, -10, -11, -12, -14, -16,
       -18, -20, -25, -30])

In [162]:
L_shs_ff = get_L_sh(15, thetas, xi)
L_shs_ff = L_shs_ff.clip(min=70) + 500
L_shs_ff 

array([2147., 1451., 1144.,  974.,  868.,  796.,  743.,  704.,  674.,
        649.,  629.,  599.,  576.,  570.,  570.,  570.,  570.])

In [144]:
3 * xi * np.cos(np.radians(thetas)) / np.tan(np.radians(thetas))

array([1475.05205159,  982.11974166,  735.28016954,  586.87854874,
        487.6968206 ,  416.64170556,  363.16671599,  321.41282708,
        287.8647451 ,  260.28557486,  237.18415333,  217.52822927,
        200.58056589,  185.80060237,  172.7829923 ,  150.86416001,
        133.06864673,  118.28021244,  105.75546451,  100.17490877,
         77.31202542])

In [163]:
L_shs_ar = get_L_sh(15, thetas, xi, h=53)
L_shs_ar = L_shs_ar.clip(min=70) + 500
L_shs_ar 

array([3665., 2462., 1902., 1580., 1372., 1227., 1121., 1039.,  974.,
        922.,  879.,  811.,  761.,  722.,  691.,  635.,  598.])

In [126]:
1/(np.radians(3)**2)

364.756261112416

In [127]:
np.radians(3)

0.05235987755982989

In [130]:
np.degrees(np.sqrt(1/900))

1.909859317102744

In [164]:
lengths_df = pd.DataFrame({'theta':thetas, 'L_sh_ff': L_shs_ff, 'L_sh_ar':L_shs_ar}).set_index('theta')
lengths_df

,L_sh_ff,L_sh_ar
theta,,
2,2147.0,3665.0
3,1451.0,2462.0
4,1144.0,1902.0
5,974.0,1580.0
6,868.0,1372.0
7,796.0,1227.0
8,743.0,1121.0
9,704.0,1039.0
10,674.0,974.0


In [178]:
minimum_angles = [4,6,10]
other_angles = set(thetas) - set(minimum_angles)

oa_df = lengths_df.loc[other_angles]

In [198]:
(-lengths_df.loc[reduced_angleset].index.values).astype(np.float)

array([ -3.,  -4.,  -5.,  -6.,  -7.,  -8.,  -9., -10., -11., -12., -14.,
       -16., -18., -20., -25., -30.])

In [200]:
lengths_df.loc[reduced_angleset]['L_sh_ff'].values.astype(np.int)

array([1451, 1144,  974,  868,  796,  743,  704,  674,  649,  629,  599,
        576,  570,  570,  570,  570])

In [197]:
reduced_angleset = set(thetas) - set([2]) 
#| set([15])
reduced_angleset

{3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 16, 18, 20, 25, 30}

In [202]:
len(reduced_angleset) * 4

64

In [204]:
64 * 48 *21

64512